![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

# PostgreSQL for Python Developers

## Census analysis with PostgreSQL

In this project, you will work with multiple different PostgreSQL adapters for Python.

You will need access to a PostgreSQL installation where you have superuser permissions. If you do not have such access elsewhere, installing to your personal workstation is a good idea.  Alternately, you might wish to use a Docker container for a self-contained installation.  See `https://hub.docker.com/_/postgres` for details on that option.  Unless you have a specific need to work with an existing installation, choosing a PostgreSQL version of 12 or higher is best.


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)
## Part 1

**Create the sample table**

This task is not particularly dependent upon the choice of adapter.  You simply want to load in the same sample data that was discussed in the lesson, located in this repository as `census-zipcodes-2018.tsv.bz2`.  It is compressed here to make the repository smaller.  Consult the Python `bz2` module or use command line tools to expand it.

The next part will work with this table.

In this part, you should also install all of the adapters discussed in this lesson.

**Solution**

The solution is exactly the same as provided in the lesson.

In [1]:
import psycopg2
user, pwd, db = 'ine_student', 'ine-password', 'ine'
host, port = 'localhost', '5432'
conn = psycopg2.connect(database=db, host=host, user=user, password=pwd, port=port)
cur = conn.cursor()

In [2]:
cur.execute('DROP TABLE IF EXISTS census_zipcode_geography;')

sql_geography = """
CREATE TABLE census_zipcode_geography (
  USPS CHAR(5) PRIMARY KEY,  -- by implication, UNIQUE NOT NULL
  ALAND BIGINT,              -- some zips are larger than 2e9 m^2
  AWATER BIGINT,
  ALAND_SQMI NUMERIC(8, 3),  -- largest zips need 5 to left of decimal
  AWATER_SQMI NUMERIC(8, 3), -- sizes with 3 digits of precision
  INTPTLAT REAL,             -- keep fields from key, although duplicative
  INTPTLONG REAL,
  location POINT             -- use geometric type for lat/lon
);
"""
cur.execute(sql_geography)

In [3]:
fields = ('USPS', 'ALAND', 'AWATER', 'ALAND_SQMI', 
          'AWATER_SQMI', 'INTPTLAT', 'INTPTLONG', 'location')

sql_insert_geo = f"""
INSERT into census_zipcode_geography ({','.join(fields)})
VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""
with open('data/census-zipcodes-2018.tsv') as fh:
    next(fh)   # discard header line
    for line in fh:
        row = line.strip().split('\t')
        row.append(f"({row[-2]}, {row[-1]})")
        cur.execute(sql_insert_geo, tuple(row))

conn.commit()

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Timing different adapters**

For the small data we are working with, all adapters will be very fast.  The speed advantages of asynchronous adapters will not be shown strongly until you work with much larger data sets than this. And especially once you have many queries "in flight* at the same time. However, your task is to time differences in performance of a relatively expensive query that utilizes nested subqueries.  A similar query is discussed passingly in the next lesson.

Are you able to measure better performance using `asyncpg` versus `pg8000` for this query? You may with to write and time scripts at the command line, since within Jupyter, you need to use the trick discussed to nest an asyncio loop within Jupyter's own loop.  More adventurous students should try much larger data sets and more complex queries that they have access to.  Moreover, you may wish to try `uvloop` rather than `asyncio` to speed up slow interfaces even more.

In [4]:
sql_nested = """
SELECT usps  
FROM census_zipcode_geography 
WHERE 10*awater_sqmi > (SELECT avg(aland_sqmi) 
                     FROM census_zipcode_geography)
AND 2*aland_sqmi > (SELECT avg(awater_sqmi)
                  FROM census_zipcode_geography)
ORDER BY location <-> point '(40.0,-105.3)'
"""

# We need this if we stay inside Jupyter
import nest_asyncio
nest_asyncio.apply()

**A possible solution**

This solution is provided as an inspiration.  More meaningful results require larger data and more complex queries, but those will depend on you obtaining such datasets, and access to PostgreSQL operating under realistic loads.

In [5]:
import pg8000
import asyncpg
import asyncio

In [6]:
%%time
conpg = pg8000.connect(database=db, host=host, user=user, password=pwd, port=port)
curpg = conpg.cursor()
curpg.execute(sql_nested)
results = curpg.fetchall()
print(f"Number of results: {len(results)}")

Number of results: 1405
CPU times: user 814 µs, sys: 4.84 ms, total: 5.65 ms
Wall time: 30.3 ms


In [7]:
%%time
conpsy = psycopg2.connect(database=db, host=host, user=user, password=pwd, port=port)
curpsy = conpsy.cursor()
curpsy.execute(sql_nested)
results = curpsy.fetchall()
print(f"Number of results: {len(results)}")

Number of results: 1405
CPU times: user 2.81 ms, sys: 925 µs, total: 3.73 ms
Wall time: 29.8 ms


In [8]:
async def query():
    results = []
    conn = await asyncpg.connect(database=db, host=host, user=user, password=pwd, port=port)
    for row in await conn.fetch(sql_nested):
        results.append(row)
    await conn.close()
    return results

In [10]:
%time   
loop = asyncio.get_event_loop()
results = loop.run_until_complete(asyncio.gather(query()))
# This nests the results an extra time because of gather()
print(f"Number of results: {len(results[0])}")                              

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
Number of results: 1405


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)